In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
print(sys.path)

%reload_ext autoreload
%autoreload 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

['/mnt/ceph/users/blyo1/projects', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python310.zip', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/lib-dynload', '', '/mnt/home/blyo1/venvs/py310/lib/python3.10/site-packages']


# hVAE

In [ ]:
import argparse
from b_models.vae import VariableConvEncoder, VariableConvDecoder, VAE, UNetDecoder

num_channels = 1
image_dims = 32
latent_dims = 5

enc = VariableConvEncoder(
    num_channels,
    image_dims,
    latent_dims, 
    channels=[32, 32, 32], 
    bias=True,
)

dec = VariableConvDecoder(
    num_channels,
    image_dims,
    latent_dims,
    channels=[32, 32, 32],  # output_channels, kernel_size, stride, padding
    output_channels=1,
    bias=True,
)

vae = VAE(
    encoder=enc,
    decoder=dec,
    kl_reduction="mean"
)